In [ ]:
library(tidyverse)
library(reshape2)
library(lme4)
library(corrplot)
library(jtools)
library(dotwhisker)
library(broom)
library(knitr)
library(ggfortify)
library(nlme)
library(simr)
library(pscl)
library(foreign) 
library("gdata")
library('ri2')
library(lmerTest)
library(binom)
library(ggthemes)
library(ggthemes)
library(lfe)
library(broom.mixed)
library(lmtest)
library(sandwich)
library(multiwayvcov)

# ANOVA Tukey Tests

In [ ]:
data_anova <- read.csv("anova.csv") 

qqnorm(data_anova$ProbaDel, main='Normality Test for Probability of Delegating')
qqline(data_anova$ProbaDel)

qqnorm(data_anova$Normalized_IRT, main='Normality Test for Expertise')
qqline(data_anova$Normalized_IRT)


aov.model <- aov(data = data_anova, ProbaDel ~ gender)
summary(aov.model)
plot(TukeyHSD(aov.model))

aov.model <- aov(data = data_anova, Normalized_IRT ~ gender, na.action=na.exclude)
summary(aov.model)
plot(TukeyHSD(aov.model))

# Estimating q

In [ ]:
data_ <- read.csv("R_paper.csv") 

model.d <- glm(ProbaDel ~ Normalized_IRT + unique_id + CodeExclu + gender, data=data_, family=binomial(logit))
vcov_firm <- cluster.vcov(model.d, select(data_, unique_id))
coeftest(model.d, vcov_firm)[1:2,]
coeftest(model.d)[1:2,]

model.d <- glm(ProbaDel ~ Normalized_IRT, data=data_, family=binomial(logit))
vcov_firm <- cluster.vcov(model.d, select(data_, unique_id))
coeftest(model.d, vcov_firm)[1:2,]
coeftest(model.d)[1:2,]

# Estimating $\varphi$

In [ ]:
data_phi <- read.csv("phi_w_gmm.csv") 

data_phi <- data_phi[!data_phi$CodeExclu %in% c('S'), ]

qqnorm(data_phi$Weight, main='Normality Test for Weights')
qqline(data_phi$Weight)

data_phi$CodeExclu <- factor(data_phi$CodeExclu)
data_phi$ExpNum <- factor(data_phi$ExpNum)
data_phi$ppi <- factor(data_phi$pi)

data_phi$full_name <- paste(data_phi$CodeExclu, data_phi$ExpNum, sep = "_")

fixed <- felm(Weight ~ pj| ExpNum + CodeExclu + ppi | 0 | full_name, data=data_phi)
print(coeftest(fixed)[1:1,])

fixed <- felm(Weight ~ pj:ppi| ExpNum + CodeExclu + ppi | 0 | full_name, data=data_phi)
print(summary(fixed))

fixed <- felm(Weight ~ pj:CodeExclu| ExpNum + CodeExclu + ppi | 0 | 0, data=data_phi)
print(summary(fixed))

# Estimating Difference

In [ ]:
lvd <- read.csv("LDvDD.csv") 
lvd <- lvd[!lvd$CodeExclu %in% c('S'), ]
lvd$full_name <- paste(lvd$CodeExclu, lvd$ExpNum, sep = "_")

lvd$CodeExcluF <- factor(lvd$CodeExclu)
lvd$ExpNum <- factor(lvd$ExpNum)

qqnorm(lvd$Estimate, main='Normality Tests for Direct and Liquid Votes')
qqline(lvd$Estimate)

summary(felm(data = lvd, Estimate ~ Cat | ExpNum + CodeQuestion + ExpNum:CodeExcluF + CodeExcluF| 0 |full_name))
coeftest(felm(data = lvd, Estimate ~ Cat | ExpNum + CodeQuestion | 0 |full_name))
summary(felm(data = lvd, Estimate ~ Cat:CodeExclu | ExpNum + CodeQuestion| 0 | full_name ))

In [ ]:
lvd <- read.csv("LDvDD_means.csv") 
lvd <- lvd[!lvd$CodeExclu %in% c('S'), ]
lvd$full_name <- paste(lvd$CodeExclu, lvd$ExpNum, sep = "_")

qqnorm(lvd$Estimate, main='Normality Tests for Direct and Liquid Average')
qqline(lvd$Estimate)

lvd$CodeExcluF <- factor(lvd$CodeExclu)
lvd$ExpNum <- factor(lvd$ExpNum)

summary(felm(data = lvd, Estimate ~ Cat | ExpNum + CodeExclu| 0 |full_name))
summary(felm(data = lvd, Estimate ~ Cat:CodeExclu | ExpNum + CodeExclu| 0 |0))
